In [218]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [219]:
df = sns.load_dataset("mpg")

X_train, y_train = df[~df["horsepower"].isna()][["displacement", "acceleration"]], df[~df["horsepower"].isna()]["horsepower"]
X_pred = df[df["horsepower"].isna()][["displacement", "acceleration"]]

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_pred)
y_pred = np.round(y_pred)
df.loc[X_pred.index, "horsepower"] = y_pred
df = pd.get_dummies(df.drop("name", axis = 1), columns = ["origin"])
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin_europe,origin_japan,origin_usa
0,18.0,8,307.0,130.0,3504,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,0,0,1


In [220]:
X, y = df.drop(["mpg"], axis = 1).values, df["mpg"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [221]:
scaler = StandardScaler()
#scaler = MinMaxScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_val = scaler.transform(X_val)
scaled_X_test = scaler.transform(X_test)

In [222]:
sigmoid = np.vectorize(lambda x: 1 / (1 + math.exp(-x)))

def seed_np(integer):
    np.random.seed(integer)

class ERegressor:
    class Net:
        def __init__(self):
            pass

        def set_input_size(self, input_size):
            self.input_size = input_size
        
        def set_weights(self, weights_and_biases):
            self.w1 = weights_and_biases[0]
            self.w2 = weights_and_biases[1]
            self.b1 = weights_and_biases[2]
            self.b2 = weights_and_biases[3]
        
        def set_mutation_sigma(self, n):
            #self.mutation_sigma = 3 - (2 - 40 / (10 + n))
            self.mutation_sigma = 2.5 - (3 - 1200 / (200 + n))
        
        def predict(self, input):
            return (self.w2.T @ sigmoid(self.w1 @ input.T + self.b1) + self.b2).reshape(-1)
        
        def __add__(self, other):
            w1 = (self.w1 + other.w1) / 2 + np.random.normal(0, self.mutation_sigma, (self.input_size, self.input_size))
            w2 = (self.w2 + other.w2) / 2 + np.random.normal(0, self.mutation_sigma, (self.input_size, 1))
            b1 = (self.b1 + other.b1) / 2 + np.random.normal(0, self.mutation_sigma, (self.input_size, 1))
            b2 = (self.b2 + other.b2) / 2 + np.random.normal(0, self.mutation_sigma, 1)

            return w1, w2, b1, b2
    
    def __init__(self, n = 100, random_state = None):
        self.nets = {}
        self.y_preds = {}
        self.best_net = None
        self.best_result = None
        self.n = n // 2 * 2
        
        for i in range(self.n):
            self.nets[i] = self.Net()

        if random_state != None:
            seed_np(random_state)
    
    def fit(self, X_train, y_train, epochs = 100):
        self.input_size = X_train.shape[1]

        for key in self.nets.keys():
            self.nets[key].set_input_size(self.input_size)

            self.nets[key].set_weights((
                np.random.uniform(-3, 3, (self.input_size, self.input_size)),
                np.random.uniform(-3, 3, (self.input_size, 1)),
                np.random.uniform(-3, 3, (self.input_size, 1)),
                np.random.uniform(-3, 3, 1))
            )

        for epoch in range(epochs):
            for key, _ in self.nets.items():
                self.y_preds[key] = self.nets[key].predict(X_train)
                self.nets[key].set_mutation_sigma(epoch)
            
            self.mean_absolute_errors = {}
            
            for key, _ in self.y_preds.items():
                self.mean_absolute_errors[key] = mean_absolute_error(y_train, self.y_preds[key])

            self.sorted_indecies = [key for key, _ in sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])]

            if self.best_result != None:
                if self.best_result[1] > sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0][1]:
                    self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                    print(f"Epoch {epoch}: MAE: {self.best_result[1]}")
            else:        
                self.best_result = sorted(self.mean_absolute_errors.items(), key = lambda x: x[1])[0]
                print(f"Epoch {epoch}: MAE: {self.best_result[1]}")

            self.best_net = self.sorted_indecies[0]
            
            for i in range(0, self.n // 2, 2):
                self.nets[self.sorted_indecies[self.n // 2 + i]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])
                self.nets[self.sorted_indecies[self.n // 2 + i + 1]].set_weights(self.nets[self.sorted_indecies[i]] + self.nets[self.sorted_indecies[i + 1]])
            

    def predict(self, X):
        return self.nets[self.best_net].predict(X)

In [223]:
for n in range(0, 1000, 10):
    #print(2.3 - (3 - 1200 / (200 + n)), end = " ")
    #print(3 - (2 - 40 / (10 + n)), end = " ")
    pass

In [224]:
regressor = ERegressor(n = 100, random_state = 42)
regressor.fit(scaled_X_train, y_train, epochs = 1000)

Epoch 0: MAE: 15.84001299310242
Epoch 1: MAE: 7.285945923052119
Epoch 2: MAE: 6.448590863228188
Epoch 4: MAE: 5.1897784768232675
Epoch 7: MAE: 4.989768557424144
Epoch 8: MAE: 4.332857643173779
Epoch 24: MAE: 4.110389103912732
Epoch 26: MAE: 3.9427008082614794
Epoch 43: MAE: 3.790552219094131
Epoch 52: MAE: 3.6476463727325266
Epoch 64: MAE: 3.5620518645558996
Epoch 70: MAE: 3.5298206838651507
Epoch 71: MAE: 3.2404421229792044
Epoch 99: MAE: 3.073567355955807
Epoch 106: MAE: 3.0448639114268827
Epoch 140: MAE: 2.8917874941117816
Epoch 150: MAE: 2.825003383829388
Epoch 154: MAE: 2.562558124977543
Epoch 330: MAE: 2.5452944191039615
Epoch 345: MAE: 2.5442006295876602
Epoch 361: MAE: 2.4799560279849113
Epoch 400: MAE: 2.427988815663674
Epoch 405: MAE: 2.4272907585639776
Epoch 411: MAE: 2.2978336872533838
Epoch 435: MAE: 2.2905575725978857
Epoch 455: MAE: 2.2868056011551916
Epoch 469: MAE: 2.2713187325647892
Epoch 485: MAE: 2.235226272956905
Epoch 531: MAE: 2.223120553092851
Epoch 535: MAE: 2.

In [225]:
y_pred = regressor.predict(scaled_X_test)

In [226]:
mean_absolute_error(y_test, y_pred)

1.6290545022529241